In [2]:
from numba import jit, cuda 
from scipy.signal import convolve2d
from scipy.integrate import ode
import numpy as np
from PIL import Image as img

In [3]:
# hàm trạng thái
def hamTrangThai(t, x, u, quanThe, n, m):
    A = taoMaTranDoiXungPerceptron(quanThe[:5])
    B = taoMaTranDoiXungPerceptron(quanThe[5:10])
    I = quanThe[10]
    x = x.reshape(n, m)
    dx = convolve2d(u, B, 'same') + convolve2d(hamDauRa(x), A, 'same') + I
    return dx.reshape(n*m)

# hàm đầu ra
def hamDauRa(x):
    return 0.5 * (np.abs(x + 1) - np.abs(x - 1))

# Tạo ma trận đối xứng từ 5 trọng số (hàm này chỉ dành riêng cho ma trận đối xứng 3x3)
def taoMaTranDoiXung(arr1):#giả sử cho mảng [1,2,3,4,5]
    arr2 = arr1[-2::-1].copy()#tạo mảng phụ [4,3,2,1]
    #arr2[0], arr2[2] = arr2[2], arr2[0]#nếu có thì sẽ đối xứng theo đường chéo phụ, nếu không thì sẽ là đối xứng tâm
    return np.append(arr1,arr2).reshape(h,h)#kết hợp hai mảng rồi chuyển về dạng ma trận 3x3

# Trong công thức biến đổi từ mạng Nơ-ron phẩn hồi sang truyền thẳng, trọng số và đầu vào , đầu ra phải được nhân với nhau theo thứ tự nhất định (xem lại công thứ)
# hàm sau đây sẽ sắp xếp lại các số trọng bộ ma trận trọng số để tương ứng với công thức.
def taoMaTranDoiXungPerceptron(arr1):
    return taoMaTranDoiXung(np.array([arr1[2],arr1[1],arr1[3],arr1[0],arr1[4]]))

# Hàm tạo số ngẫu nhiên từ -9.99 đến 9.99
def ngauNhien():
    return np.random.randint(-999,1000)/100.

# Hàm tính sai số
def saiSo(d,y):
    return np.sum(0.5*(d-y)**2)

# Bán kính lân cận của nơ ron tế bào
r = 1

# kích thước của ma trận điều khiển và ma trận phản hồi (hai ma trận này luôn vuông)
h = 2*r + 1 # h = 3

class CNN:

    slts = 11 # số lượng trọng số trong quần thể

    # Quần thể trọng số (gồm 11 số)
    quanThe = np.array([ngauNhien() for i in range(slts)])
    # quanThe = np.array([0.,0.,0.,0.,2.,0.,0.,0.,0.,0.,-1.])

    # ma trận ảnh đầu vào u
    u = np.array(img.open('images/u1.png').convert('RGB'), dtype=float)[:,:,0]/255.0*2.0-1.0

    # kích thước ma trận đầu vào, ma trận đầu ra và ma trận trạng thái
    m, n = u.shape
    print(m,n)

    # ma trận đầu ra mong muốn d
    d = np.array(img.open('images/d1.png').convert('RGB'), dtype=float)[:,:,0]/255.0*2.0-1.0

    # hai giá trị ban đầu cho ptvp gồm x0 = [ma trận toàn số 0] với t0 = 0.0 và t1 để tính đầu ra x1 (t1 != t0)
    x0, t0, t1 = u*0., 0., 0.2

    # ma trận đầu ra y (y được tính theo x)
    y = hamDauRa(x0)

    # sai số cho phép
    delta_W0 = 0.00000000

    # Tốc độ học của thuật toán
    alpha = 0.01

cnn = CNN()

# In ra bộ ma trận trọng số:
def print_A_B_I():
    print(f'A = {taoMaTranDoiXung(cnn.quanThe[:5])}')
    print(f'B = {taoMaTranDoiXung(cnn.quanThe[5:10])}')
    print(f'I = {cnn.quanThe[10]}')

# Hàm để lấy trọng số từ file trongSoBac1ToiUu rồi lưu vào cnn.quanThe
def getQuanThe(i):# lấy bộ trọng số thứ i
    A = np.loadtxt(f'TrongSoBac1ToiUu/TrongSo{i}/A.txt')
    B = np.loadtxt(f'TrongSoBac1ToiUu/TrongSo{i}/B.txt')
    I = np.loadtxt(f'TrongSoBac1ToiUu/TrongSo{i}/I.txt')
    cnn.quanThe = np.array([A[0][0],A[0][1],A[0][2],A[1][0],A[1][1],B[0][0],B[0][1],B[0][2],B[1][0],B[1][1],I])

# tạo thêm biên gồm toàn số 0 bao quanh ma trận a để tránh tràn biên
def them_bien(a):
    b = np.zeros(shape=[a.shape[0]+2,a.shape[1]+2])
    for i in range(1,a.shape[0]+1):
        for j in range(1,a.shape[1]+1):
            b[i][j] = a[i-1][j-1]
    return b

# tính x, tính y, tính wij và cập trọng số
#@jit(target_backend='cuda')
def capNhatTrongSo():
    ptvp.set_initial_value(cnn.x0.flatten(), 0.)
    ptvp.set_f_params(cnn.u, cnn.quanThe, cnn.n, cnn.m)
    assert ptvp.successful()
    x = (ptvp.integrate(cnn.t1))[:].reshape(cnn.n, cnn.m)
    cnn.y = hamDauRa(x)
    cnn.y = np.sign(cnn.y)
    y = them_bien(cnn.y) # thêm biên 0 cho y để tránh tràn biên
    u = them_bien(cnn.u) # thêm biên 0 cho u để tránh tràn biên
    delta_W = 0.
    for i in range(1,cnn.m+1):
        for j in range(1,cnn.n+1):
            Ys = np.array([
                y[i-1][j] + y[i+1][j], 
                y[i][j-1] + y[i][j+1],
                y[i-1][j-1] + y[i+1][j+1],
                y[i-1][j+1] + y[i+1][j-1],
                y[i][j],
                u[i-1][j] + u[i+1][j], 
                u[i][j-1] + u[i][j+1],
                u[i-1][j-1] + u[i+1][j+1],
                u[i-1][j+1] + u[i+1][j-1],
                u[i][j],
                1            
                ])
            norm_n = np.sum(Ys**2)
            wij = cnn.alpha * (cnn.d[i-1][j-1] - y[i][j]) / norm_n
            delta_W += wij
            if(wij != 0.):
                for ts in range(cnn.slts):
                    cnn.quanThe[ts] += wij * Ys[ts]
    return delta_W

ptvp = ode(hamTrangThai) 
ptvp.set_integrator('vode') # thuật toán được sử dụng để giải ptvp, đọc thêm trong tài liệu
ptvp.set_initial_value(cnn.x0.flatten(), 0.) # lệnh flatten dùng để biến đổi ma trận về dạng mảng (đầu vào của ode phải là một số hoặc mảng)

getQuanThe(10)
print_A_B_I()
print(f'u = {cnn.u}')
print(f'd = {cnn.d}')

ptvp.set_f_params(cnn.u, cnn.quanThe, cnn.n, cnn.m)
assert ptvp.successful()
x = (ptvp.integrate(cnn.t1))[:].reshape(cnn.n, cnn.m)
y = hamDauRa(x)
cnn.y = np.sign(y)
print(f'y = {cnn.y}')
print(f'Sai số: E = {saiSo(cnn.d,cnn.y)}')

8 8
A = [[ 1.28 -1.05 -2.05]
 [-4.2  -6.98 -4.2 ]
 [-2.05 -1.05  1.28]]
B = [[-2.83 -3.14  0.7 ]
 [ 0.27  9.64  0.27]
 [ 0.7  -3.14 -2.83]]
I = 4.55
u = [[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1.  1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]
d = [[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1.  1.]
 [ 1.  1. -1.  1.  1.  1. -1.  1.]
 [ 1.  1. -1.  1. -1. -1. -1.  1.]
 [ 1.  1. -1.  1. -1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]
y = [[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1. -1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1

In [20]:
print('Bộ trọng số ban đầu: ')
print_A_B_I()
# vòng lặp lớn
loop = 10000 #số lần chạy thử tối đa
count = 0 # đếm số lần chạy
while(True):
    # kiểm tra ptvp ODE
    if(not ptvp.successful()):
        print('Lỗi phương trình vi phân')
        break

    # đếm số lần chạy
    if(count>loop):
        print('Đạt đến số vòng lặp tối đa')
        break
    else:
        print(f'Lần chạy thứ {count + 1}')
    delta_W = capNhatTrongSo()

    # kiểm tra sai số
    if(delta_W <= cnn.delta_W0):
        print('Đạt đến mức sai số cho phép')
        break

    count+=1

    print_A_B_I()
    print(f'Sai số hiện tại: w = {delta_W}')
    print(f'Sai số: E = {saiSo(cnn.d,cnn.y)}')

print('Bộ trọng sau khi tối ưu: ')
print_A_B_I()
print(f'Sai số: E = {saiSo(cnn.d,cnn.y)}')

Bộ trọng số ban đầu: 
A = [[ 1.28 -1.05 -2.05]
 [-4.2  -6.98 -4.2 ]
 [-2.05 -1.05  1.28]]
B = [[-2.83 -3.14  0.7 ]
 [ 0.27  9.64  0.27]
 [ 0.7  -3.14 -2.83]]
I = 4.55
Lần chạy thứ 0
Đạt đến mức sai số cho phép
Bộ trọng sau khi tối ưu: 
A = [[ 1.29487762 -1.06062434 -2.04460386]
 [-4.19312238 -6.98743881 -4.19312238]
 [-2.04460386 -1.06062434  1.29487762]]
B = [[-2.81740809 -3.13837105  0.70014747]
 [ 0.27459191  9.64629595  0.27459191]
 [ 0.70014747 -3.13837105 -2.81740809]]
I = 4.547957330602493
Sai số: E = 16.0
